# Expense recorder for my Journey Journal
This won't quite work the way I intended

In [40]:
import requests
from flask import jsonify
import json
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import pandas as pd
from datetime import datetime

# Set base URL for API calls
url = 'http://localhost:5000/'

# We'll use this to append to the base URL for specific API calls
url_part = ''

# We'll reserve the payload for POST/PUT methods generally
payload = {}

# We don't have any specific headers, but this can be used for authentication if required
headers = {}

Navigate to a trip

In [36]:
# Ask for a trip ID (this will be replaced with a dropdown in the app)
trip_id = input("Enter trip id: ")

# Get the trip information
url_part = 'trips/' + str(trip_id)
response = requests.request("GET", url + url_part, headers=headers, data=payload)
trip = response.json()

# Get the trip information and format a trip title
destination = trip['destination']
start_date = datetime.strptime(trip['start_date'], '%a, %d %b %Y %H:%M:%S %Z') # Convert to datetime object
end_date = datetime.strptime(trip['end_date'], '%a, %d %b %Y %H:%M:%S %Z') # Convert to datetime object

trip_title = destination + ': ' + start_date.strftime('%b %#d, %Y') + ' - ' + end_date.strftime('%b %#d, %Y')

# Get the trip's expenses
url_part = 'trips/' + str(trip_id) + '/expenses'
response = requests.request("GET", url + url_part, headers=headers, data=payload)
expenses = response.json()

In [57]:
import dash_table

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        'Search a different trip:',
        dcc.Input(id='input-on-submit', type='text')
    ]),
    html.H2(children=trip_title, style={'textAlign':'left'}),
    html.H1(children='EXPENSES', style={'textAlign':'left'}),
    html.Div([
        html.Button('Add an expense', id='submit-val', n_clicks=0),
        " ",
        html.Button('Edit an expense', id='submit-val2', n_clicks=0)
    ]),
    dash_table.DataTable(
        id='expenses-table',
        columns=[{'name': header, 'id': header} for header in expenses[0].keys()],
        data=expenses
    )
])

if __name__ == '__main__':
    app.run(debug=True)